In [6]:
import torch
import torch.nn as nn
from transformers import (
    DebertaV2Tokenizer, DebertaV2Model,
    AutoTokenizer, AutoModel
)       

/home/yanai-lab/qiu-y/miniconda3/envs/vad/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
vid_mllm_ckpt = "/export/space0/qiu-y/model/deberta-v3-base"
device = "cuda:0"
cls_names = [
        'normal', 'abuse', 'arrest', 'arson', 'assault',
        'burglary', 'explosion', 'fighting', 'roadAccidents',
        'robbery', 'shooting', 'shoplifting', 'stealing', 'vandalism']
cls_num = len(cls_names)

learnable_prompt_len = 10

Class names: ['normal', 'abuse', 'arrest', 'arson', 'assault', 'burglary', 'explosion', 'fighting', 'roadAccidents', 'robbery', 'shooting', 'shoplifting', 'stealing', 'vandalism'], total 14 classes.


In [32]:
mllm = AutoModel.from_pretrained(vid_mllm_ckpt).to(device)
tokenizer = AutoTokenizer.from_pretrained(vid_mllm_ckpt)
model_hidden = mllm.config.hidden_size  # 768
# seq_max = min(256, mllm.config.max_position_embeddings)
seq_max = 77


/home/yanai-lab/qiu-y/miniconda3/envs/vad/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [44]:
encoded = tokenizer(
            cls_names,
            max_length=int(seq_max - 2 * learnable_prompt_len),
            padding='max_length',
            truncation=True,
            return_tensors='pt',
        )
word_tokens = encoded["input_ids"]
attn_mask = encoded['attention_mask'].to(device)
print(word_tokens.shape, attn_mask.shape)  

torch.Size([14, 57]) torch.Size([14, 57])


In [45]:
token_embed = mllm.get_input_embeddings()
word_emb = token_embed(word_tokens.to(device))
word_emb.shape

torch.Size([14, 57, 768])

In [46]:
prefix = nn.Parameter(torch.randn(cls_num, learnable_prompt_len, model_hidden) * 0.02).to(device)
suffix = nn.Parameter(torch.randn(cls_num, learnable_prompt_len, model_hidden) * 0.02).to(device)
print(prefix.shape, suffix.shape)

torch.Size([14, 10, 768]) torch.Size([14, 10, 768])


In [47]:
inputs_embeds = torch.cat([prefix, word_emb, suffix], dim=1)
 
print(inputs_embeds.shape)

torch.Size([14, 77, 768])


In [48]:
ones = torch.ones(cls_num, learnable_prompt_len, device=device, dtype=attn_mask.dtype)
print(ones.shape)
attn_mask = torch.cat([ones, attn_mask, ones], dim=1)  
print(attn_mask.shape)

torch.Size([14, 10])
torch.Size([14, 77])


In [53]:
# 送入模型（模型内部统一加位置编码；不要再手工加位置）
out = mllm(inputs_embeds=inputs_embeds, attention_mask=attn_mask)
out.last_hidden_state.shape

torch.Size([14, 77, 768])